# Toyota GR Cup: Tire Degradation Model Training

**Objective**: Predict tire degradation rate based on driving aggression metrics

**Dataset**: 
- 2,545 laps from Toyota GR Cup racing series
- 21 normalized features (brake pressure, G-forces, steering, throttle, speed, RPM)
- Target: `tire_degradation_rate` (seconds per lap)

**Models to Compare**:
1. Linear Regression (baseline)
2. Ridge Regression (L2 regularization)
3. Lasso Regression (L1 regularization)
4. Random Forest
5. Gradient Boosting
6. XGBoost (GPU-ready)

**Evaluation Metrics**: R², MAE, RMSE, Explained Variance

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
from pathlib import Path
from datetime import datetime

# ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import xgboost as xgb

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print(f"XGBoost version: {xgb.__version__}")
print(f"XGBoost GPU support: {xgb.build_info()['USE_CUDA']}")

## 2. Load Preprocessed Data

In [ ]:
# Load features and target
features_path = Path('../ml_data/features_normalized.csv')
target_path = Path('../ml_data/target_degradation.csv')

X = pd.read_csv(features_path)
y = pd.read_csv(target_path)['tire_degradation_rate']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")
print(f"\nTarget statistics:")
print(y.describe())

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(y, bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Tire Degradation Rate (seconds/lap)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Target Variable Distribution')
axes[0].axvline(y.mean(), color='red', linestyle='--', label=f'Mean: {y.mean():.3f}')
axes[0].legend()

# Box plot
axes[1].boxplot(y, vert=True)
axes[1].set_ylabel('Tire Degradation Rate (seconds/lap)')
axes[1].set_title('Target Variable Box Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Target variable range: [{y.min():.3f}, {y.max():.3f}]")
print(f"Outliers (> Q3 + 1.5*IQR): {sum(y > y.quantile(0.75) + 1.5 * (y.quantile(0.75) - y.quantile(0.25)))}")

In [ ]:
# Feature correlation heatmap
plt.figure(figsize=(16, 14))
correlation_matrix = X.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Heatmap', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# Identify highly correlated features (>0.9)
high_corr = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            high_corr.append((correlation_matrix.columns[i], 
                            correlation_matrix.columns[j], 
                            correlation_matrix.iloc[i, j]))

if high_corr:
    print("\nHighly correlated features (|r| > 0.9):")
    for feat1, feat2, corr in high_corr:
        print(f"  {feat1} <-> {feat2}: {corr:.3f}")
else:
    print("\nNo highly correlated features found (threshold: |r| > 0.9)")

In [ ]:
# Feature vs Target correlation
feature_target_corr = X.corrwith(y).sort_values(ascending=False)

plt.figure(figsize=(12, 8))
feature_target_corr.plot(kind='barh', color='steelblue')
plt.xlabel('Correlation with Tire Degradation Rate')
plt.title('Feature Importance: Correlation with Target Variable', fontsize=14, pad=20)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 5 positively correlated features:")
print(feature_target_corr.head())
print("\nTop 5 negatively correlated features:")
print(feature_target_corr.tail())

## 4. Train/Test Split

In [ ]:
# Split data (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Train/Test ratio: {X_train.shape[0]/X_test.shape[0]:.2f}")

## 5. Model Training and Evaluation

We'll train 6 different models and compare their performance.

In [ ]:
# Define models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.01, random_state=42),
    'Random Forest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=15,
        min_samples_split=5,
        random_state=42,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        tree_method='auto',  # Change to 'gpu_hist' on Windows with CUDA
        n_jobs=-1
    )
}

print("Models configured. Ready for training.")
print(f"\nNote: XGBoost tree_method set to 'auto' (CPU).")
print("On Windows with GPU, change to 'gpu_hist' for CUDA acceleration.")

In [ ]:
# Train all models and collect results
results = {}
trained_models = {}

print("Training models...\n")
print("="*80)

for name, model in models.items():
    print(f"\n{name}:")
    print("-" * 40)
    
    # Train model
    start_time = datetime.now()
    model.fit(X_train, y_train)
    train_time = (datetime.now() - start_time).total_seconds()
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_ev = explained_variance_score(y_test, y_test_pred)
    
    # Cross-validation (5-fold)
    cv_scores = cross_val_score(
        model, X_train, y_train, cv=5, 
        scoring='r2', n_jobs=-1
    )
    
    # Store results
    results[name] = {
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'test_ev': test_ev,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'train_time': train_time,
        'predictions': y_test_pred
    }
    trained_models[name] = model
    
    # Print results
    print(f"Train R²:     {train_r2:.4f}")
    print(f"Test R²:      {test_r2:.4f}")
    print(f"Test MAE:     {test_mae:.4f}")
    print(f"Test RMSE:    {test_rmse:.4f}")
    print(f"Explained Var: {test_ev:.4f}")
    print(f"CV R² (5-fold): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print(f"Training Time: {train_time:.2f}s")
    
    # Check for overfitting
    if train_r2 - test_r2 > 0.1:
        print("⚠️  Warning: Potential overfitting detected")

print("\n" + "="*80)
print("\nAll models trained successfully!")

## 6. Model Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df[['test_r2', 'test_mae', 'test_rmse', 'cv_mean', 'cv_std', 'train_time']]
comparison_df = comparison_df.sort_values('test_r2', ascending=False)

print("\nModel Performance Comparison:")
print("="*100)
print(comparison_df.to_string())
print("\n" + "="*100)

best_model_name = comparison_df.index[0]
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Test R²: {comparison_df.loc[best_model_name, 'test_r2']:.4f}")
print(f"   Test MAE: {comparison_df.loc[best_model_name, 'test_mae']:.4f}")
print(f"   Test RMSE: {comparison_df.loc[best_model_name, 'test_rmse']:.4f}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# R² Score
comparison_df['test_r2'].plot(kind='barh', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_xlabel('R² Score')
axes[0, 0].set_title('Test R² Score Comparison', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='x')

# MAE
comparison_df['test_mae'].plot(kind='barh', ax=axes[0, 1], color='coral')
axes[0, 1].set_xlabel('Mean Absolute Error')
axes[0, 1].set_title('Test MAE Comparison (Lower is Better)', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='x')

# RMSE
comparison_df['test_rmse'].plot(kind='barh', ax=axes[1, 0], color='lightgreen')
axes[1, 0].set_xlabel('Root Mean Squared Error')
axes[1, 0].set_title('Test RMSE Comparison (Lower is Better)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Training Time
comparison_df['train_time'].plot(kind='barh', ax=axes[1, 1], color='plum')
axes[1, 1].set_xlabel('Training Time (seconds)')
axes[1, 1].set_title('Training Time Comparison', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 7. Predictions vs Actual (Best Model)

In [ ]:
# Get best model predictions
best_predictions = results[best_model_name]['predictions']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot: Predicted vs Actual
axes[0].scatter(y_test, best_predictions, alpha=0.6, edgecolor='black', s=50)
axes[0].plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Tire Degradation Rate', fontsize=12)
axes[0].set_ylabel('Predicted Tire Degradation Rate', fontsize=12)
axes[0].set_title(f'{best_model_name}: Predictions vs Actual', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residual plot
residuals = y_test - best_predictions
axes[1].scatter(best_predictions, residuals, alpha=0.6, edgecolor='black', s=50)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Tire Degradation Rate', fontsize=12)
axes[1].set_ylabel('Residuals (Actual - Predicted)', fontsize=12)
axes[1].set_title(f'{best_model_name}: Residual Plot', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Residual statistics
print(f"\nResidual Statistics ({best_model_name}):")
print(f"Mean: {residuals.mean():.6f}")
print(f"Std Dev: {residuals.std():.6f}")
print(f"Min: {residuals.min():.6f}")
print(f"Max: {residuals.max():.6f}")

## 8. Feature Importance Analysis

In [ ]:
# Feature importance for tree-based models
tree_models = ['Random Forest', 'Gradient Boosting', 'XGBoost']

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for idx, model_name in enumerate(tree_models):
    if model_name in trained_models:
        model = trained_models[model_name]
        
        # Get feature importance
        if hasattr(model, 'feature_importances_'):
            importance = model.feature_importances_
            feature_importance_df = pd.DataFrame({
                'feature': X.columns,
                'importance': importance
            }).sort_values('importance', ascending=True)
            
            # Plot
            axes[idx].barh(feature_importance_df['feature'], 
                          feature_importance_df['importance'],
                          color='steelblue')
            axes[idx].set_xlabel('Feature Importance', fontsize=11)
            axes[idx].set_title(f'{model_name}\nFeature Importance', 
                               fontsize=12, fontweight='bold')
            axes[idx].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Print top 10 features for best model (if tree-based)
if best_model_name in tree_models:
    best_model = trained_models[best_model_name]
    feature_importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Most Important Features ({best_model_name}):")
    print("="*60)
    print(feature_importance_df.head(10).to_string(index=False))

## 9. Save Best Model

In [ ]:
# Create models directory if it doesn't exist
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Save best model
best_model = trained_models[best_model_name]
model_filename = f'tire_degradation_model_{best_model_name.lower().replace(" ", "_")}.pkl'
model_path = models_dir / model_filename

joblib.dump(best_model, model_path)
print(f"✅ Best model saved: {model_path}")

# Save all models
print("\nSaving all trained models...")
for name, model in trained_models.items():
    filename = f'tire_degradation_model_{name.lower().replace(" ", "_")}.pkl'
    filepath = models_dir / filename
    joblib.dump(model, filepath)
    print(f"  ✅ {name} -> {filepath.name}")

# Save model metadata
metadata = {
    'best_model': best_model_name,
    'best_test_r2': results[best_model_name]['test_r2'],
    'best_test_mae': results[best_model_name]['test_mae'],
    'best_test_rmse': results[best_model_name]['test_rmse'],
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'training_samples': X_train.shape[0],
    'test_samples': X_test.shape[0],
    'features': list(X.columns),
    'all_models': {name: {
        'test_r2': results[name]['test_r2'],
        'test_mae': results[name]['test_mae'],
        'test_rmse': results[name]['test_rmse'],
        'cv_mean': results[name]['cv_mean'],
        'cv_std': results[name]['cv_std']
    } for name in results.keys()}
}

import json
metadata_path = models_dir / 'model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n✅ Model metadata saved: {metadata_path}")

## 10. Model Loading Example (For Future Use)

In [ ]:
# Example: Load trained model
loaded_model = joblib.load(model_path)
print(f"✅ Model loaded from: {model_path}")

# Test prediction on first 5 test samples
sample_predictions = loaded_model.predict(X_test.head())
print(f"\nSample predictions (first 5 test samples):")
for i, (actual, pred) in enumerate(zip(y_test.head(), sample_predictions)):
    print(f"  Sample {i+1}: Actual={actual:.4f}, Predicted={pred:.4f}, Error={abs(actual-pred):.4f}")

## 11. Summary and Next Steps

In [ ]:
print("\n" + "="*80)
print("TRAINING COMPLETE - SUMMARY")
print("="*80)

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Test R²: {results[best_model_name]['test_r2']:.4f}")
print(f"   Test MAE: {results[best_model_name]['test_mae']:.4f} seconds/lap")
print(f"   Test RMSE: {results[best_model_name]['test_rmse']:.4f} seconds/lap")
print(f"   CV R² (5-fold): {results[best_model_name]['cv_mean']:.4f} ± {results[best_model_name]['cv_std']:.4f}")

print(f"\n📊 Models Trained: {len(trained_models)}")
print(f"📁 Models Saved: {len(list(models_dir.glob('*.pkl')))}")
print(f"📈 Training Samples: {X_train.shape[0]}")
print(f"📉 Test Samples: {X_test.shape[0]}")

print("\n" + "="*80)
print("NEXT STEPS:")
print("="*80)
print("""
1. 🖥️  Transfer to Windows for GPU training:
   - Follow database transfer instructions in docs/DATABASE_TRANSFER.md
   - Install GPU requirements: pip install -r requirements-gpu.txt
   - Update XGBoost tree_method to 'gpu_hist' for CUDA acceleration
   
2. 🔧 Hyperparameter Tuning:
   - Use GridSearchCV/RandomizedSearchCV on best models
   - Focus on: n_estimators, max_depth, learning_rate, min_samples_split
   
3. 🔬 Feature Engineering:
   - Create interaction features (e.g., brake_pressure * lateral_g)
   - Add polynomial features
   - Test lap_in_stint transformations (log, sqrt)
   
4. 🎯 Model Ensembling:
   - Stack top 3 models (VotingRegressor)
   - Weighted averaging based on CV scores
   
5. 🚀 Production Script:
   - Convert this notebook to src/train_model.py
   - Add command-line arguments
   - Create src/predict.py for inference
   
6. 📝 Documentation:
   - Document model performance in README.md
   - Create model card with metrics and limitations
""")
print("="*80)